* This notebook is heavily inspired by the introductory workshop on PostGIS.
* The same tasks are done with DuckDB.


In [1]:
! pip install duckdb

In [2]:
import duckdb
import pandas as pd

In [3]:
con = duckdb.connect()

In [4]:
con.install_extension('spatial')
con.load_extension('spatial')

## Download the data

In [5]:
!wget http://s3.cleverelephant.ca/postgis-workshop-2020.zip


--2024-02-26 12:05:22--  http://s3.cleverelephant.ca/postgis-workshop-2020.zip
Resolving s3.cleverelephant.ca (s3.cleverelephant.ca)... 16.182.71.216, 16.182.97.240, 16.182.67.160, ...
Connecting to s3.cleverelephant.ca (s3.cleverelephant.ca)|16.182.71.216|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22547324 (22M) [application/zip]
Saving to: ‘postgis-workshop-2020.zip’

postgis-workshop-20 100%[===================>]  21.50M  50.9MB/s    in 0.4s    

2024-02-26 12:05:23 (50.9 MB/s) - ‘postgis-workshop-2020.zip’ saved [22547324/22547324]



In [6]:
!unzip postgis-workshop-2020.zip

Archive:  postgis-workshop-2020.zip
   creating: postgis-workshop/
   creating: postgis-workshop/printing/
  inflating: postgis-workshop/printing/nyc_data_dictionary.pdf  
  inflating: postgis-workshop/printing/postgis-workshop-exercises.docx  
  inflating: postgis-workshop/printing/postgis-workshop-exercises.pdf  
  inflating: postgis-workshop/printing/nyc_data_dictionary.docx  
   creating: postgis-workshop/data/
  inflating: postgis-workshop/data/nyc_homicides.dbf  
  inflating: postgis-workshop/data/nyc_neighborhoods.shx  
  inflating: postgis-workshop/data/nyc_neighborhoods.shp  
  inflating: postgis-workshop/data/nyc_census_blocks.shx  
  inflating: postgis-workshop/data/nyc_streets.prj  
  inflating: postgis-workshop/data/nyc_subway_stations.dbf  
  inflating: postgis-workshop/data/nyc_census_blocks.shp  
  inflating: postgis-workshop/data/nyc_subway_stations.shx  
  inflating: postgis-workshop/data/nyc_census_blocks.dbf  
  inflating: postgis-workshop/data/nyc_subway_stations.s

In [7]:
!ls

postgis-workshop  postgis-workshop-2020.zip  sample_data


In [8]:
con.sql('SELECT * FROM ST_Read("postgis-workshop/data/nyc_neighborhoods.shp");')

┌───────────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│   BORONAME    │         NAME         │                                     geom                                      │
│    varchar    │       varchar        │                                   geometry                                    │
├───────────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│ Brooklyn      │ Bensonhurst          │ POLYGON ((582771.4257198056 4495167.427365481, 584651.2943549604 4497541.64…  │
│ Manhattan     │ East Village         │ POLYGON ((585508.7534890148 4509691.267208001, 586826.3570590394 4508984.18…  │
│ Manhattan     │ West Village         │ POLYGON ((583263.2776595836 4509242.626023987, 583276.8199068634 4509378.82…  │
│ The Bronx     │ Throggs Neck         │ POLYGON ((597640.0090688139 4520272.719938631, 597647.7457808304 4520617.82…  │
│ The Bronx     │ Wakefield-Will

In [9]:
con.sql('CREATE TABLE IF NOT EXISTS nyc_neighborhoods AS SELECT * FROM ST_Read("postgis-workshop/data/nyc_neighborhoods.shp");')

## Simple SQL

In [10]:
con.sql('SELECT * FROM nyc_neighborhoods;')

┌───────────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│   BORONAME    │         NAME         │                                     geom                                      │
│    varchar    │       varchar        │                                   geometry                                    │
├───────────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│ Brooklyn      │ Bensonhurst          │ POLYGON ((582771.4257198056 4495167.427365481, 584651.2943549604 4497541.64…  │
│ Manhattan     │ East Village         │ POLYGON ((585508.7534890148 4509691.267208001, 586826.3570590394 4508984.18…  │
│ Manhattan     │ West Village         │ POLYGON ((583263.2776595836 4509242.626023987, 583276.8199068634 4509378.82…  │
│ The Bronx     │ Throggs Neck         │ POLYGON ((597640.0090688139 4520272.719938631, 597647.7457808304 4520617.82…  │
│ The Bronx     │ Wakefield-Will

In [11]:
con.sql('SELECT name FROM nyc_neighborhoods;')

┌──────────────────────────┐
│           NAME           │
│         varchar          │
├──────────────────────────┤
│ Bensonhurst              │
│ East Village             │
│ West Village             │
│ Throggs Neck             │
│ Wakefield-Williamsbridge │
│ Auburndale               │
│ Battery Park             │
│ Carnegie Hill            │
│ Mariners Harbor          │
│ Rossville                │
│     ·                    │
│     ·                    │
│     ·                    │
│ Yorkville                │
│ Chinatown                │
│ Bayside                  │
│ Coney Island             │
│ Corona                   │
│ Red Hook                 │
│ Douglastown-Little Neck  │
│ Whitestone               │
│ Steinway                 │
│ Rosebank                 │
├──────────────────────────┤
│   129 rows (20 shown)    │
└──────────────────────────┘

In [12]:
con.sql("SELECT name, ST_TRANSFORM(geom,'EPSG:26918','EPSG:4326') FROM nyc_neighborhoods;")


┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│         NAME         │                         st_transform(geom, 'EPSG:26918', 'EPSG:4326')                         │
│       varchar        │                                           geometry                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ Bensonhurst          │ POLYGON ((40.603177255343375 -74.02166685535772, 40.624372593164594 -73.99913147363611, 40.…  │
│ East Village         │ POLYGON ((40.73372002435467 -73.9873400213324, 40.727213268064766 -73.9718363715135, 40.721…  │
│ West Village         │ POLYGON ((40.729909197333484 -74.01398852243662, 40.731134619728365 -74.01381006286464, 40.…  │
│ Throggs Neck         │ POLYGON ((40.827674726653015 -73.84204449268826, 40.830782040132526 -73.84189867432876, 40.…  │
│ Wakefield-Williams…  │ POLYGON

In [13]:
con.sql("SELECT name FROM nyc_neighborhoods WHERE boroname  = 'Brooklyn';")

┌──────────────────────────┐
│           NAME           │
│         varchar          │
├──────────────────────────┤
│ Bensonhurst              │
│ Bay Ridge                │
│ Boerum Hill              │
│ Cobble Hill              │
│ Downtown                 │
│ Sunset Park              │
│ Borough Park             │
│ East Brooklyn            │
│ Flatbush                 │
│ Park Slope               │
│ Williamsburg             │
│ Canarsie                 │
│ Greenwood                │
│ Gravesend-Sheepshead Bay │
│ Dyker Heights            │
│ Brownsville              │
│ Bushwick                 │
│ Fort Green               │
│ Mapleton-Flatlands       │
│ Bedford-Stuyvesant       │
│ Carroll Gardens          │
│ Coney Island             │
│ Red Hook                 │
├──────────────────────────┤
│         23 rows          │
└──────────────────────────┘

In [14]:
# Question: How many neighborhoods are in Brooklyn?
con.sql("SELECT COUNT(*) FROM nyc_neighborhoods WHERE boroname  = 'Brooklyn';")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           23 │
└──────────────┘

In [15]:
# What is the number of letters in the names of all the  neighborhoods in Brooklyn?
con.sql("SELECT length(name) FROM nyc_neighborhoods WHERE boroname  = 'Brooklyn';")

┌────────────────┐
│ length("name") │
│     int64      │
├────────────────┤
│             11 │
│              9 │
│             11 │
│             11 │
│              8 │
│             11 │
│             12 │
│             13 │
│              8 │
│             10 │
│             12 │
│              8 │
│              9 │
│             24 │
│             13 │
│             11 │
│              8 │
│             10 │
│             18 │
│             18 │
│             15 │
│             12 │
│              8 │
├────────────────┤
│    23 rows     │
└────────────────┘

In [16]:
# What is the average number of letters and standard deviation of number of letters in the names of all the neighborhoods in Brooklyn?
con.sql("SELECT avg(length(name)), stddev_pop(length(name)) FROM nyc_neighborhoods WHERE boroname  = 'Brooklyn';")

┌─────────────────────┬────────────────────────────┐
│ avg(length("name")) │ stddev_pop(length("name")) │
│       double        │           double           │
├─────────────────────┼────────────────────────────┤
│   11.73913043478261 │         3.8246044558694345 │
└─────────────────────┴────────────────────────────┘

In [17]:
#What is the average number of letters in the names of all the neighborhoods in New York City, reported by borough?
con.sql("SELECT avg(length(name)),BORONAME  FROM nyc_neighborhoods GROUP BY BORONAME;")

┌─────────────────────┬───────────────┐
│ avg(length("name")) │   BORONAME    │
│       double        │    varchar    │
├─────────────────────┼───────────────┤
│   11.73913043478261 │ Brooklyn      │
│  12.041666666666666 │ The Bronx     │
│  11.666666666666666 │ Queens        │
│  11.821428571428571 │ Manhattan     │
│  12.291666666666666 │ Staten Island │
└─────────────────────┴───────────────┘

In [18]:
# How many records are in the nyc_streets table?
con.sql('CREATE TABLE IF NOT EXISTS nyc_streets AS SELECT * FROM ST_Read("postgis-workshop/data/nyc_streets.shp");')
con.sql("SELECT COUNT(*)  FROM nyc_streets;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        19090 │
└──────────────┘

In [19]:
con.sql("SELECT *  FROM nyc_streets;")

┌───────┬─────────────────┬─────────┬───────────────┬──────────────────────────────────────────────────────────────────┐
│  ID   │      NAME       │ ONEWAY  │     TYPE      │                               geom                               │
│ int64 │     varchar     │ varchar │    varchar    │                             geometry                             │
├───────┼─────────────────┼─────────┼───────────────┼──────────────────────────────────────────────────────────────────┤
│     1 │ Shore Pky S     │ NULL    │ residential   │ LINESTRING (586785.4767897038 4492901.0014554765, 586898.23200…  │
│     2 │ NULL            │ NULL    │ footway       │ LINESTRING (586645.0073625665 4504977.750360583, 586664.224899…  │
│     3 │ Avenue O        │ NULL    │ residential   │ LINESTRING (586750.3019977848 4496109.72213903, 586837.3726880…  │
│     4 │ Walsh Ct        │ NULL    │ residential   │ LINESTRING (586728.695515043 4497971.05313857, 586886.35822565…  │
│     5 │ NULL            │ NULL

In [20]:
con.sql("SELECT name, starts_with(name, 'B')  FROM nyc_streets;")

┌─────────────────┬──────────────────────────┐
│      NAME       │ starts_with("name", 'B') │
│     varchar     │         boolean          │
├─────────────────┼──────────────────────────┤
│ Shore Pky S     │ false                    │
│ NULL            │ NULL                     │
│ Avenue O        │ false                    │
│ Walsh Ct        │ false                    │
│ NULL            │ NULL                     │
│ Avenue Z        │ false                    │
│ Dank Ct         │ false                    │
│ Cumberland Walk │ false                    │
│ Cumberland Walk │ false                    │
│ NULL            │ NULL                     │
│  ·              │  ·                       │
│  ·              │  ·                       │
│  ·              │  ·                       │
│ 90th Ave        │ false                    │
│ 133rd Ave       │ false                    │
│ Aberdeen Rd     │ false                    │
│ Bonnie Ln       │ true                     │
│ Phroane Ave

In [21]:
# How many streets in NYC start with ‘B’?
con.sql("SELECT COUNT(*)  FROM nyc_streets WHERE starts_with(name, 'B') = true;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1282 │
└──────────────┘

In [22]:
# How many streets in NYC start with ‘B’? (second method)
con.sql("SELECT Count(*)  FROM nyc_streets WHERE name LIKE 'B%';")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1282 │
└──────────────┘

In [23]:
# Population of NYC; reading shapefile into table & printing the first rows of the table
con.sql('CREATE TABLE IF NOT EXISTS nyc_census_blocks AS SELECT * FROM ST_Read("postgis-workshop/data/nyc_census_blocks.shp");')
con.sql("SELECT * FROM nyc_census_blocks LIMIT 5;")

┌─────────────────┬────────────┬────────────┬───┬────────────┬────────────┬───────────────┬──────────────────────┐
│      BLKID      │ POPN_TOTAL │ POPN_WHITE │ … │ POPN_ASIAN │ POPN_OTHER │   BORONAME    │         geom         │
│     varchar     │   int64    │   int64    │   │   int64    │   int64    │    varchar    │       geometry       │
├─────────────────┼────────────┼────────────┼───┼────────────┼────────────┼───────────────┼──────────────────────┤
│ 360850009001000 │         97 │         51 │ … │          5 │          8 │ Staten Island │ POLYGON ((577856.5…  │
│ 360850020011000 │         66 │         52 │ … │          7 │          5 │ Staten Island │ POLYGON ((578620.7…  │
│ 360850040001000 │         62 │         14 │ … │         25 │          3 │ Staten Island │ POLYGON ((577227.2…  │
│ 360850074001000 │        137 │         92 │ … │         13 │         20 │ Staten Island │ POLYGON ((579037.0…  │
│ 360850096011000 │        289 │        230 │ … │         32 │         27 │ Stat

In [24]:
# What is the population of NYC?
con.sql("SELECT sum(POPN_TOTAL) AS population FROM nyc_census_blocks;")

┌────────────┐
│ population │
│   int128   │
├────────────┤
│    8174377 │
└────────────┘

In [25]:
# What is the population of ‘The Bronx’?
con.sql("SELECT sum(POPN_TOTAL) FROM nyc_census_blocks WHERE BORONAME = 'The Bronx';")


┌─────────────────┐
│ sum(POPN_TOTAL) │
│     int128      │
├─────────────────┤
│         1384453 │
└─────────────────┘

In [26]:
# How many neighborhoods are in each borough?
con.sql('SELECT COUNT(*),BORONAME  FROM nyc_neighborhoods GROUP BY BORONAME;')

┌──────────────┬───────────────┐
│ count_star() │   BORONAME    │
│    int64     │    varchar    │
├──────────────┼───────────────┤
│           23 │ Brooklyn      │
│           24 │ The Bronx     │
│           30 │ Queens        │
│           28 │ Manhattan     │
│           24 │ Staten Island │
└──────────────┴───────────────┘

In [27]:
# For each borough in NYC, what is percentage of the population is “white”?
con.sql("SELECT BORONAME , 100.0 * sum(POPN_WHITE)/sum(POPN_TOTAL) FROM nyc_census_blocks GROUP BY BORONAME;")

┌───────────────┬───────────────────────────────────────────────┐
│   BORONAME    │ ((100.0 * sum(POPN_WHITE)) / sum(POPN_TOTAL)) │
│    varchar    │                    double                     │
├───────────────┼───────────────────────────────────────────────┤
│ Brooklyn      │                             42.80117379326865 │
│ The Bronx     │                            27.905822732877173 │
│ Queens        │                             39.72207739459101 │
│ Staten Island │                              72.8942034860154 │
│ Manhattan     │                             57.44930394804628 │
└───────────────┴───────────────────────────────────────────────┘

## Geometries

In [28]:
# What is the area of the ‘West Village’ neighborhood?
con.sql("SELECT ST_Area(geom) FROM nyc_neighborhoods WHERE name = 'West Village';")

┌────────────────────┐
│   st_area(geom)    │
│       double       │
├────────────────────┤
│ 1044614.5296485956 │
└────────────────────┘

In [29]:
# What is the geometry type of ‘Pelham St’? The length?
con.sql("SELECT ST_GeometryType(geom) AS geom_type, name, geom from nyc_streets WHERE name = 'Pelham St';")


┌──────────────────────┬───────────┬───────────────────────────────────────────────────────────────────────────────────┐
│      geom_type       │   NAME    │                                       geom                                        │
│ enum('point', 'lin…  │  varchar  │                                     geometry                                      │
├──────────────────────┼───────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ LINESTRING           │ Pelham St │ LINESTRING (585323.7613806158 4507140.08120791, 585318.1174625934 4507190.08686…  │
└──────────────────────┴───────────┴───────────────────────────────────────────────────────────────────────────────────┘

In [30]:
con.sql("SELECT ST_Length(geom) AS geom_length, name from nyc_streets WHERE name = 'Pelham St';")

┌───────────────────┬───────────┐
│    geom_length    │   NAME    │
│      double       │  varchar  │
├───────────────────┼───────────┤
│ 50.32314951660229 │ Pelham St │
└───────────────────┴───────────┘

In [31]:
# create a table from nyc_subway_stations shapefile in the database
con.sql('CREATE TABLE IF NOT EXISTS nyc_subway_stations AS SELECT * FROM ST_Read("postgis-workshop/data/nyc_subway_stations.shp");')

con.sql('SELECT * from nyc_subway_stations LIMIT 3;')

┌──────────┬────────┬──────────────┬──────────┬───┬───────────┬─────────┬─────────┬─────────┬──────────────────────┐
│ OBJECTID │   ID   │     NAME     │ ALT_NAME │ … │ TRANSFERS │  COLOR  │ EXPRESS │ CLOSED  │         geom         │
│  double  │ double │   varchar    │ varchar  │   │  varchar  │ varchar │ varchar │ varchar │       geometry       │
├──────────┼────────┼──────────────┼──────────┼───┼───────────┼─────────┼─────────┼─────────┼──────────────────────┤
│      1.0 │  376.0 │ Cortlandt St │ NULL     │ … │ R,W       │ YELLOW  │ NULL    │ NULL    │ POINT (583521.8544…  │
│      2.0 │    2.0 │ Rector St    │ NULL     │ … │ 1         │ RED     │ NULL    │ NULL    │ POINT (583324.4866…  │
│      3.0 │    1.0 │ South Ferry  │ NULL     │ … │ 1         │ RED     │ NULL    │ NULL    │ POINT (583304.1823…  │
├──────────┴────────┴──────────────┴──────────┴───┴───────────┴─────────┴─────────┴─────────┴──────────────────────┤
│ 3 rows                                                        

In [32]:
# What is the GeoJSON representation of ‘Broad St’ subway station?
con.sql("SELECT ST_AsGeoJSON(geom), name FROM nyc_subway_stations WHERE name = 'Broad St';")

┌──────────────────────────────────────────────────────────────────────┬──────────┐
│                          st_asgeojson(geom)                          │   NAME   │
│                               varchar                                │ varchar  │
├──────────────────────────────────────────────────────────────────────┼──────────┤
│ {"type":"Point","coordinates":[583571.9059213118,4506714.341192182]} │ Broad St │
└──────────────────────────────────────────────────────────────────────┴──────────┘

In [33]:
con.sql("SELECT ST_Length(geom) / 1000 AS street_length_km, name from nyc_streets;")

┌──────────────────────┬─────────────────┐
│   street_length_km   │      NAME       │
│        double        │     varchar     │
├──────────────────────┼─────────────────┤
│   0.7974398579946621 │ Shore Pky S     │
│  0.14879975102119924 │ NULL            │
│    2.312304024906042 │ Avenue O        │
│  0.16039575340314952 │ Walsh Ct        │
│   0.0877145235596008 │ NULL            │
│   3.3619916036595883 │ Avenue Z        │
│  0.17333277056192278 │ Dank Ct         │
│  0.14735137946342827 │ Cumberland Walk │
│  0.19715823234272525 │ Cumberland Walk │
│ 0.024186507314031266 │ NULL            │
│            ·         │  ·              │
│            ·         │  ·              │
│            ·         │  ·              │
│    1.610084884554142 │ 90th Ave        │
│  0.31142915293481516 │ 133rd Ave       │
│   1.2715409645153237 │ Aberdeen Rd     │
│  0.07416356832871748 │ Bonnie Ln       │
│   0.2153371272826522 │ Phroane Ave     │
│  0.20217578676797485 │ 186th Ln        │
│    0.2222

In [34]:
# What is the total length of streets (in kilometers) in New York City?
con.sql("SELECT sum(ST_Length(geom) / 1000) from nyc_streets;")

┌───────────────────────────────┐
│ sum((st_length(geom) / 1000)) │
│            double             │
├───────────────────────────────┤
│            10418.265759654627 │
└───────────────────────────────┘

In [35]:
# What is the area of Manhattan in acres? (using nyc_census_blocks table)
con.sql('CREATE TABLE IF NOT EXISTS nyc_census_blocks AS SELECT * FROM ST_Read("postgis-workshop/data/nyc_census_blocks.shp");')

con.sql("SELECT sum(ST_Area(geom) / 4047) FROM nyc_census_blocks WHERE BORONAME = 'Manhattan';")

┌─────────────────────────────┐
│ sum((st_area(geom) / 4047)) │
│           double            │
├─────────────────────────────┤
│          14601.398721554813 │
└─────────────────────────────┘

In [36]:
# What is the area of Manhattan in acres? (using nyc_neighborhoods table)
con.sql("SELECT sum(ST_Area(geom) / 4047) FROM nyc_neighborhoods WHERE BORONAME = 'Manhattan';")

┌─────────────────────────────┐
│ sum((st_area(geom) / 4047)) │
│           double            │
├─────────────────────────────┤
│          13965.320122391191 │
└─────────────────────────────┘

In [37]:
# What is the most westerly subway station?
con.sql('DESCRIBE nyc_subway_stations;')


┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ OBJECTID    │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ ID          │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ NAME        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ ALT_NAME    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ CROSS_ST    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ LONG_NAME   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ LABEL       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ BOROUGH     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ NGHBHD      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ ROUTES      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ TRANSFERS   │ VARCHAR     │ YES     │ NULL    

In [38]:
# What is the most westerly subway station?
con.sql('SELECT ST_X(geom), name from nyc_subway_stations ORDER BY ST_X(geom) LIMIT 1;')

┌───────────────────┬─────────────┐
│    st_x(geom)     │    NAME     │
│      double       │   varchar   │
├───────────────────┼─────────────┤
│ 563292.1172580556 │ Tottenville │
└───────────────────┴─────────────┘

In [39]:
# What is the most easterly subway station?
con.sql('SELECT ST_X(geom), name from nyc_subway_stations ORDER BY ST_X(geom) DESC LIMIT 1;')

┌───────────────────┬──────────────┐
│    st_x(geom)     │     NAME     │
│      double       │   varchar    │
├───────────────────┼──────────────┤
│ 605412.4548804507 │ Far Rockaway │
└───────────────────┴──────────────┘

## Spatial Relationships

In [40]:
con.sql("SELECT * from nyc_subway_stations;")

┌──────────┬────────┬───────────────────┬───┬──────────────┬─────────┬─────────┬──────────────────────┐
│ OBJECTID │   ID   │       NAME        │ … │    COLOR     │ EXPRESS │ CLOSED  │         geom         │
│  double  │ double │      varchar      │   │   varchar    │ varchar │ varchar │       geometry       │
├──────────┼────────┼───────────────────┼───┼──────────────┼─────────┼─────────┼──────────────────────┤
│      1.0 │  376.0 │ Cortlandt St      │ … │ YELLOW       │ NULL    │ NULL    │ POINT (583521.8544…  │
│      2.0 │    2.0 │ Rector St         │ … │ RED          │ NULL    │ NULL    │ POINT (583324.4866…  │
│      3.0 │    1.0 │ South Ferry       │ … │ RED          │ NULL    │ NULL    │ POINT (583304.1823…  │
│      4.0 │  125.0 │ 138th St          │ … │ GREEN        │ NULL    │ NULL    │ POINT (590250.1059…  │
│      5.0 │  126.0 │ 149th St          │ … │ GREEN        │ express │ NULL    │ POINT (590454.7399…  │
│      6.0 │   45.0 │ 149th St          │ … │ RED-GREEN    │ exp

In [41]:
# What is geometry of Broad Street subway station?
con.sql("SELECT geom, name from nyc_subway_stations WHERE name = 'Broad St';")

┌─────────────────────────────────────────────┬──────────┐
│                    geom                     │   NAME   │
│                  geometry                   │ varchar  │
├─────────────────────────────────────────────┼──────────┤
│ POINT (583571.9059213118 4506714.341192182) │ Broad St │
└─────────────────────────────────────────────┴──────────┘

In [ ]:
# What is geometry of Broad Street subway station?
con.sql("SELECT ST_GeometryType(geom), geom, name from nyc_subway_stations WHERE name = 'Broad St';")

┌─────────────────────────────────────────────────────────────┬─────────────────────────────────────────────┬──────────┐
│                    st_geometrytype(geom)                    │                    geom                     │   NAME   │
│ enum('point', 'linestring', 'polygon', 'multipoint', 'mul…  │                  geometry                   │ varchar  │
├─────────────────────────────────────────────────────────────┼─────────────────────────────────────────────┼──────────┤
│ POINT                                                       │ POINT (583571.9059213118 4506714.341192182) │ Broad St │
└─────────────────────────────────────────────────────────────┴─────────────────────────────────────────────┴──────────┘

In [ ]:
con.sql("SELECT typeof(geom) from nyc_subway_stations WHERE name = 'Broad St';")

┌──────────────┐
│ typeof(geom) │
│   varchar    │
├──────────────┤
│ GEOMETRY     │
└──────────────┘

In [ ]:
# What is the well-known binary (WKB) of Broad Street subway station?
con.sql("SELECT ST_AsHEXWKB(geom) from nyc_subway_stations WHERE name = 'Broad St';")

┌────────────────────────────────────────────┐
│             st_ashexwkb(geom)              │
│                  varchar                   │
├────────────────────────────────────────────┤
│ 01010000000EEBD4CF27CF2141BC17D69516315141 │
└────────────────────────────────────────────┘

In [ ]:
# What subway station record matches that geometry?
con.sql("SELECT name from nyc_subway_stations WHERE ST_Equals(geom, ST_GeomFromHEXWKB('01010000000EEBD4CF27CF2141BC17D69516315141'));")

┌──────────┐
│   NAME   │
│ varchar  │
├──────────┤
│ Broad St │
└──────────┘

In [ ]:
# What is the well-known text (WKT) of Broad Street subway station?
con.sql("SELECT ST_AsText(geom) from nyc_subway_stations WHERE name = 'Broad St';")

┌─────────────────────────────────────────────┐
│               st_astext(geom)               │
│                   varchar                   │
├─────────────────────────────────────────────┤
│ POINT (583571.9059213118 4506714.341192182) │
└─────────────────────────────────────────────┘

In [ ]:
# What is the type of well-known text (WKT) of Broad Street subway station?
con.sql("SELECT typeof(ST_AsText(geom)) from nyc_subway_stations WHERE name = 'Broad St';")

┌─────────────────────────┐
│ typeof(st_astext(geom)) │
│         varchar         │
├─────────────────────────┤
│ VARCHAR                 │
└─────────────────────────┘

In [ ]:
con.sql("SELECT * from nyc_neighborhoods;")

┌───────────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│   BORONAME    │         NAME         │                                     geom                                      │
│    varchar    │       varchar        │                                   geometry                                    │
├───────────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│ Brooklyn      │ Bensonhurst          │ POLYGON ((582771.4257198056 4495167.427365481, 584651.2943549604 4497541.64…  │
│ Manhattan     │ East Village         │ POLYGON ((585508.7534890148 4509691.267208001, 586826.3570590394 4508984.18…  │
│ Manhattan     │ West Village         │ POLYGON ((583263.2776595836 4509242.626023987, 583276.8199068634 4509378.82…  │
│ The Bronx     │ Throggs Neck         │ POLYGON ((597640.0090688139 4520272.719938631, 597647.7457808304 4520617.82…  │
│ The Bronx     │ Wakefield-Will

In [ ]:
# What neighborhood intersects that subway station?
con.sql("SELECT name, boroname  from nyc_neighborhoods WHERE ST_Intersects(geom, ST_GeomFromHEXWKB('01010000000EEBD4CF27CF2141BC17D69516315141'));")


┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Financial District │ Manhattan │
└────────────────────┴───────────┘

In [ ]:
# What streets are within 10 meters of Broad Street subway station?

con.sql("SELECT name from nyc_streets WHERE ST_DWithin(geom, ST_GeomFromHEXWKB('01010000000EEBD4CF27CF2141BC17D69516315141'), 10);")


┌───────────┐
│   NAME    │
│  varchar  │
├───────────┤
│ Wall St   │
│ Broad St  │
│ Nassau St │
└───────────┘

In [ ]:
# What streets are within 10 meters of Broad Street subway station? (second method)
con.sql("SELECT name from nyc_streets WHERE ST_DWithin(geom, ST_GeomFromText('POINT(583571.9059213118 4506714.341192182)'), 10);")

┌───────────┐
│   NAME    │
│  varchar  │
├───────────┤
│ Wall St   │
│ Broad St  │
│ Nassau St │
└───────────┘

In [ ]:
# What is the well-known text for the street 'Atlantic Commons'?
con.sql("SELECT ST_AsText(geom) from nyc_streets WHERE name = 'Atlantic Commons';")

┌───────────────────────────────────────────────────────────────────────────────────────┐
│                                    st_astext(geom)                                    │
│                                        varchar                                        │
├───────────────────────────────────────────────────────────────────────────────────────┤
│ LINESTRING (586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098) │
└───────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
# What neighborhood and borough is 'POINT(586782 4504202)' in?
con.sql("SELECT BORONAME, NAME FROM nyc_neighborhoods WHERE ST_Intersects(ST_GeomFromText('POINT(583571.9059213118 4506714.341192182)'), geom);")

┌───────────┬────────────────────┐
│ BORONAME  │        NAME        │
│  varchar  │      varchar       │
├───────────┼────────────────────┤
│ Manhattan │ Financial District │
└───────────┴────────────────────┘

In [51]:
# How many people live within 50 meters of 'POINT(586782 4504202)'?

con.sql("SELECT Sum(POPN_TOTAL) FROM nyc_census_blocks WHERE ST_DWithin(ST_GeomFromText('POINT(586782 4504202)'), geom, 50);")

┌─────────────────┐
│ sum(POPN_TOTAL) │
│     int128      │
├─────────────────┤
│             896 │
└─────────────────┘

In [57]:
con.sql("SELECT * FROM nyc_streets WHERE name = 'Columbus Cir';")

┌───────┬──────────────┬─────────┬─────────┬───────────────────────────────────────────────────────────────────────────┐
│  ID   │     NAME     │ ONEWAY  │  TYPE   │                                   geom                                    │
│ int64 │   varchar    │ varchar │ varchar │                                 geometry                                  │
├───────┼──────────────┼─────────┼─────────┼───────────────────────────────────────────────────────────────────────────┤
│ 18686 │ Columbus Cir │ yes     │ primary │ LINESTRING (585880.4300287147 4513529.824197933, 585883.3962299527 4513…  │
└───────┴──────────────┴─────────┴─────────┴───────────────────────────────────────────────────────────────────────────┘

In [56]:
# How far apart are 'Columbus Cir' and 'Fulton Ave'?
con.sql("SELECT ST_Distance(a.geom, b.geom) FROM nyc_streets a , nyc_streets b WHERE a.name = 'Fulton Ave' AND b.name = 'Columbus Cir';")

┌─────────────────────────────┐
│ st_distance(a.geom, b.geom) │
│           double            │
├─────────────────────────────┤
│           9184.786200076896 │
└─────────────────────────────┘

## Spatial Joins

## Resources:

* [Presentation Deck of PostGIS workshop](https://docs.google.com/presentation/d/1qYXdeCIymLl32uoAHvAPrp1r-hK-_4Z8InG7sHEo6vc/edit)

* [download and unzip in GColab](https://buomsoo-kim.github.io/colab/2020/05/04/Colab-downloading-files-from-web-2.md/)

* [Reprojection of geometry column in duckdb](https://tech.marksblogg.com/duckdb-gis-spatial-extension.html)

